In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from pydantic import BaseModel
import importlib
import shutil
from utils import utils
importlib.reload(utils)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [7]:
subject = '수학'
room_id = '674e58163925f28f6caf4fa0'

file_keys = utils.get_items('pagecall-text', f'{subject}/{room_id}')
utils.download_items('pagecall-text', file_keys, './downloads')

In [8]:
raw_data = utils.merge_files('./downloads')
shutil.rmtree('./downloads')

In [30]:
T_extracted_data = utils.extract_speaker(raw_data, speaker='teacher')
S_extracted_data = utils.extract_speaker(raw_data, speaker='student')

T_splited_data = utils.split_sentences(T_extracted_data)
S_splited_data = utils.split_sentences(S_extracted_data)
T_splited_data[:10], S_splited_data[:10]

([{'idx': 0, 'text': '목소리 잘 들릴까요?'},
  {'idx': 1, 'text': '안녕하세요.'},
  {'idx': 2, 'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
  {'idx': 3, 'text': '잘 보여요?'},
  {'idx': 4, 'text': '잠깐만요.'},
  {'idx': 5, 'text': '혹시 전에 선택으로 과외 해본 적 있어요?'},
  {'idx': 6, 'text': '아 정말요?'},
  {'idx': 7, 'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?'},
  {'idx': 8, 'text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요'},
  {'idx': 9, 'text': '아 그거 괜찮아요'}],
 [{'idx': 0, 'text': '감사합니다.'},
  {'idx': 1, 'text': '네 안녕하세요'},
  {'idx': 2, 'text': '네 네'},
  {'idx': 3, 'text': '네, 같이 볼게요.'},
  {'idx': 4, 'text': '어떤 걸 보여줄지 모르겠어가지고..'},
  {'idx': 5, 'text': '아니 근데..'},
  {'idx': 6, 'text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..'},
  {'idx': 7, 'text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..'},
  {'idx': 8, 'text': '그..'},
  {'idx': 9, 'text': '모르는 게 너무 많아가지고..'}])

In [31]:

for item in T_splited_data:
    time_list = []
    text_list = item['text'].split()
    for text in text_list:
        for data in T_extracted_data:
            if text in data['text']:
                time_list.append(data['time_range'])
                # print('original:', data['text'])
                data['text'] = data['text'].replace(text, "", 1).strip()
                # print('text:', text)
                # print('remaining:', data['text'])
                break
            continue
    item['time_range'] = set(time_list)
    print(item['text'], set(time_list))

목소리 잘 들릴까요? {'4m 54.5s ~ 4m 58.3s'}
안녕하세요. {'4m 58.5s ~ 5m 0.6s'}
잘 들리고 그러면 필기 한번 확인해 볼게요. {'5m 0.7s ~ 5m 6.2s'}
잘 보여요? {'5m 9.7s ~ 5m 12.5s'}
잠깐만요. {'5m 12.5s ~ 5m 12.6s'}
혹시 전에 선택으로 과외 해본 적 있어요? {'5m 23.3s ~ 5m 35.5s'}
아 정말요? {'5m 35.5s ~ 5m 38.1s'}
저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요? {'5m 53.9s ~ 5m 57.2s', '5m 38.8s ~ 5m 52.8s'}
이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요 {'5m 59.9s ~ 6m 23.0s'}
아 그거 괜찮아요 {'5m 59.9s ~ 6m 23.0s'}
같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요. {'5m 59.9s ~ 6m 23.0s', '6m 23.6s ~ 6m 33.3s'}
그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요. {'6m 34.6s ~ 6m 41.5s'}
그래서 일단은 지금 하반기니까, 하반기 거를 일단 하고, 기말고사 얼마 안 남았으니까 샌쩍 같이 본 다음에 시간이 남으면 상반기 걸 같이 볼게요 {'6m 42.5s ~ 6m 48.8s', '6m 49.6s ~ 7m 13.3s'}
그리고 제가 이거 답을 한번 봤는데 일단 풍거는 거의 다 맞았던 것 같아요 {'6m 49.6s ~ 7m 13.3s'}
네 그래서 풍거는 잘 푸는 것 같고 별표 위주로 한번 보면 될 것 같고 그 다음에 이게 지금 교재가 깨끗한 상태잖아요. {'6m 49.6s ~ 7m 13.3s', '7m 14.1s ~ 7m 16.3s'}
근데 하윤 학생이 뭘 틀렸는지 제가 모르니까. {'7m 17.3s ~ 7m 20.2s'}
이거를 하윤 학생이 그때그때 말해주면 좋을 것 같아요. {'7m 21.4s ~ 7m

In [32]:

for item in S_splited_data:
    time_list = []
    text_list = item['text'].split()
    for text in text_list:
        for data in S_extracted_data:
            if text in data['text']:
                time_list.append(data['time_range'])
                # print('original:', data['text'])
                data['text'] = data['text'].replace(text, "", 1).strip()
                # print('text:', text)
                # print('remaining:', data['text'])
                break
            continue
    item['time_range'] = set(time_list)
    print(item['text'], set(time_list))

감사합니다. {'0m 0.0s ~ 0m 2.4s'}
네 안녕하세요 {'4m 41.8s ~ 5m 10.4s'}
네 네 {'4m 41.8s ~ 5m 10.4s'}
네, 같이 볼게요. {'5m 35.0s ~ 5m 55.5s'}
어떤 걸 보여줄지 모르겠어가지고.. {'6m 4.0s ~ 6m 33.1s'}
아니 근데.. {'6m 4.0s ~ 6m 33.1s'}
근데 솔직히 볼 수 있는 게 몇 개 없어가지고.. {'6m 4.0s ~ 6m 33.1s'}
거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요.. {'6m 4.0s ~ 6m 33.1s'}
그.. {'6m 4.0s ~ 6m 33.1s'}
모르는 게 너무 많아가지고.. {'6m 4.0s ~ 6m 33.1s'}
네.. {'6m 4.0s ~ 6m 33.1s'}
아니 진짜 충격부모일 수도 있어요. {'6m 4.0s ~ 6m 33.1s'}
네.. {'6m 33.2s ~ 6m 33.2s'}
그런가요? {'6m 37.4s ~ 7m 1.3s'}
네. {'7m 1.4s ~ 7m 1.7s'}
네. {'7m 6.7s ~ 7m 6.7s'}
네. {'7m 9.7s ~ 7m 9.7s'}
네. {'7m 40.5s ~ 7m 55.5s'}
괜찮아요. {'7m 55.6s ~ 8m 8.9s'}
음 음 음 {'8m 11.2s ~ 8m 33.3s'}
응. {'8m 43.8s ~ 8m 45.7s'}
응. {'8m 50.4s ~ 8m 50.5s'}
네. {'9m 14.3s ~ 9m 14.4s'}
아. {'9m 53.6s ~ 10m 13.3s'}
아. {'10m 14.2s ~ 10m 14.2s'}
오오 {'10m 33.2s ~ 11m 1.2s'}
허성고요? {'11m 8.8s ~ 11m 33.1s'}
몰라요. {'11m 35.1s ~ 11m 35.5s'}
네 네 네 네 네 {'11m 40.0s ~ 12m 4.6s'}
네 네 {'12m 15.5s ~ 12m 38.9s'}
네 네 네 네 네 네 네 {'12m 46.7s ~ 13m 15.0s'}
네 네 네 네 네 

In [33]:
T_splited_data

[{'idx': 0, 'text': '목소리 잘 들릴까요?', 'time_range': {'4m 54.5s ~ 4m 58.3s'}},
 {'idx': 1, 'text': '안녕하세요.', 'time_range': {'4m 58.5s ~ 5m 0.6s'}},
 {'idx': 2,
  'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.',
  'time_range': {'5m 0.7s ~ 5m 6.2s'}},
 {'idx': 3, 'text': '잘 보여요?', 'time_range': {'5m 9.7s ~ 5m 12.5s'}},
 {'idx': 4, 'text': '잠깐만요.', 'time_range': {'5m 12.5s ~ 5m 12.6s'}},
 {'idx': 5,
  'text': '혹시 전에 선택으로 과외 해본 적 있어요?',
  'time_range': {'5m 23.3s ~ 5m 35.5s'}},
 {'idx': 6, 'text': '아 정말요?', 'time_range': {'5m 35.5s ~ 5m 38.1s'}},
 {'idx': 7,
  'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?',
  'time_range': {'5m 38.8s ~ 5m 52.8s', '5m 53.9s ~ 5m 57.2s'}},
 {'idx': 8,
  'text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요',
  'time_range': {'5m 59.9s ~ 6m 23.0s'}},
 {'idx': 9, 'text': '아 그거 괜찮아요', 'time_range': {'5m 59.9s ~ 6m 23.0s'}},
 {'idx': 10,
  'text': '같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.',
  'time_range': {'5m 59.9s ~ 6m 23.0s', '6m 23.6s ~ 6m 33.3s'}},
 {'idx': 11,
  'text'

In [ ]:
# chunks_with_overlap = utils.split_with_overlap(splited_data, chunk_size=30, overlap=5)
# chunks_with_overlap

[[{'idx': 0, 'text': '감사합니다.', 'time_range': {'0m 0.0s ~ 0m 2.4s'}},
  {'idx': 1, 'text': '네 안녕하세요', 'time_range': {'4m 41.8s ~ 5m 10.4s'}},
  {'idx': 2, 'text': '네 네', 'time_range': {'4m 41.8s ~ 5m 10.4s'}},
  {'idx': 3, 'text': '네, 같이 볼게요.', 'time_range': {'5m 35.0s ~ 5m 55.5s'}},
  {'idx': 4,
   'text': '어떤 걸 보여줄지 모르겠어가지고..',
   'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 5, 'text': '아니 근데..', 'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 6,
   'text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..',
   'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 7,
   'text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..',
   'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 8, 'text': '그..', 'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 9, 'text': '모르는 게 너무 많아가지고..', 'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 10, 'text': '네..', 'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 11,
   'text': '아니 진짜 충격부모일 수도 있어요.',
   'time_range': {'6m 4.0s ~ 6m 33.1s'}},
  {'idx': 12, 'text': '네..', 'time_range': {'6m 33.2s ~

In [26]:
from prompt import question_analyzer
from prompt import blue_question_analyzer

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)
user ='student'

importlib.reload(blue_question_analyzer)
system_prompt = blue_question_analyzer.QuestionAnalyzer(subject,user).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
# question_analysis_results = chain.batch([
#     {"user_message": chunk} for chunk in chunks_with_overlap
# ])
input = []


AuthenticationError: Error code: 401 - {'error': {'message': 'Authentication Fails (no such user)', 'type': 'authentication_error', 'param': None, 'code': 'invalid_request_error'}}

In [7]:
question_indices = utils.extract_question_indices(question_analysis_results)
question_context = utils.get_question_context(splited_data, question_indices)
question_context

[{'idx': 1,
  'question': '잘 들려요?',
  'context': '어 안녕하세요 잘 들려요? 네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요 일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요 네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요. 빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요. 근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
 {'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 {'idx': 13,
  'question': '혹시 아까도 소리 들렸나요?',
  'context': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의

In [8]:
from prompt import question_classifier

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_classifier)
system_prompt = question_classifier.QuestionClassifier(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in question_context
])

In [9]:
for (res,item) in zip(question_classifier_results, question_context):
    item['result'] = res
question_context

[{'idx': 1,
  'question': '잘 들려요?',
  'context': '어 안녕하세요 잘 들려요? 네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요 일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요 네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요. 빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요. 근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.',
  'result': 'False'},
 {'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠',
  'result': 'True'},
 {'idx': 13,
  'question': '혹시 아까도 소리 들렸나요?',
  'context': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거

In [10]:
final_feed = [{'idx': item['idx'], 'question': item['question'], 'context': item['context']} for item in question_context if item['result'] == 'True']
final_feed

[{'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 {'idx': 17,
  'question': '무슨 말인지 이해가 좀 되나요?',
  'context': '그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠 무슨 말인지 이해가 좀 되나요? 그러면은 지금 이번 문제에서 연준학생이 해본거 보면 해외 한거를 보면은 지금 보면은 이 선본 ai를 이렇게 그어놓고 지금 필기 공유가 되나요 여기 보면은 여기 40도 있고 그 다음에 여기 25도 이렇게 있잖아요 이렇게 되는게 아니고 이게 이렇게 뇌접원 이렇게 그어지고 그러면은 지금 제가 갖고 있는 여기 선분 두 개가 같으니까 요거 두 개 길이 같단 말이에요 그러면은 여기를 X라고 하고 여기

In [11]:
from prompt import digging

llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(digging)
system_prompt = digging.Digging(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
results = chain.batch([
    {"user_message": chunk} for chunk in final_feed
])

In [12]:
results = [json.loads(item.strip('```json').strip('```').strip()) for item in results]

for (res, item) in zip(results, final_feed):
    item['result'] = res['result']
    item['answer'] = res['answer']
    item['reason'] = res['reason']
final_feed

[{'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠',
  'result': '이해점검형 질문',
  'answer': '네, 내심은 삼각형의 내접원의 중심이고, 각 변에서의 거리가 같은 점이에요.',
  'reason': "질문은 학생이 내심의 정의를 알고 있는지 확인하는 것이며, '네' 또는 '아니오'로 대답할 수 있는 형태입니다. 따라서 이는 학생의 이해도를 확인하기 위한 이해점검형 질문입니다."},
 {'idx': 17,
  'question': '무슨 말인지 이해가 좀 되나요?',
  'context': '그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠 무슨 말인지 이해가 좀 되나요? 그러면은 지금 이번 문제에서 연준학생이 

In [13]:
with open(f"{room_id}.txt", "w", encoding="utf-8-sig") as file:
    json.dump(final_feed, file, ensure_ascii=False, indent=4)